### Toronto Neighborhood ###

In [1]:
!pip install bs4
!pip install lxml

In [2]:
import pandas as pd
import requests
from lxml import html
from bs4 import BeautifulSoup

In [38]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content, "lxml")
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df=pd.DataFrame(df[0])

  
  
Deleting the boroughs that are not assigned

In [39]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)




Current Postal code list on Wikipedia page is already rouped.  There is no need to set a neighborhood name as the same as the borough. We have to replace the slash with the comma

In [34]:
df = df.replace({'/': ','}, regex=True)

In [35]:
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [36]:
df

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [40]:
print(df.shape)

(103, 3)
